
# <span style="color:#336699">Image processing on images obtained through STAC</span>
<hr style="border:2px solid #0077b9;">
<div style="text-align: left;">
    <a href="https://nbviewer.jupyter.org/github/davidoesch/satromo-dev/blob/main/codegallery/jupyter/Python/stac/stac-image-processing.ipynb"><img src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg" align="center"/></a>
</div>




<b>Abstract.</b> This Jupyter Notebook describes how to search for data <em>SwissEO</em> products in data.geo.admin.ch's catalog through the STAC service. Then it shows how to use Python libraries to perform some image processing. It starts by computing the Normalized Difference Vegetation Index (NDVI) based on the red and near-infrared spectral bands. Next, it demonstrates a threshold analysis based on the computed NDVI. Lastly, it computes the NDVI difference for images from two diffrent dates.
</div>

<br/>
<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>For an introduction to the SpatioTemporal Asset Catalog (STAC) with the data.geo.admin.ch infrastructure, please, refer to the following Jupyter Notebook:</b>
    <div style="margin-left: 10px; margin-right: 10px">
    <a href="./stac-introduction.ipynb" target="_blank">Introduction to the SpatioTemporal Asset Catalog (STAC)</a>.
    </div>
</div>


# STAC Client API
<hr style="border:1px solid #0077b9;">

For running the examples in this Jupyter Notebook you will need to install the [STAC client for Python](https://github.com/brazil-data-cube/stac.py). To install it from PyPI using `pip`, use the following command:

In [8]:
!pip install pystac-client

In [9]:
!pip install rasterio shapely matplotlib

In order to access the funcionalities of the client API, you should import the `pystac_client` package, as follows:

In [10]:
import pystac_client

After that, you can check the installed `pystac_client` package version:

In [2]:
pystac_client.__version__

'0.8.5'

Then, create a `STAC` object attached to the data.geo.admin.ch STAC service:

In [3]:
service = pystac_client.Client.open('https://data.geo.admin.ch/api/stac/v0.9/')

Due to the "Swisstopo finish" STAC implementation,  we need to add the conformance classes: 

In [4]:
service.add_conforms_to("COLLECTIONS")
service.add_conforms_to("ITEM_SEARCH")

# Searching for Sentinel-2 Images
<hr style="border:1px solid #0077b9;">

We are going to use the STAC `search` API to look for images from collection named `ch.swisstopo.swisseo_s2-sr_v100`. This collection is a temporal composite from Sentinel-2/MSI surface reflectance data. Let's define a search box with the following bounds: $x_{min} = 7.364515,$, $x_{max} = 7.535312$, $y_{min} = -46.868545,$, $y_{max} = -46.999127$. Besides that, the period of interest ranges from August 1st, 2018 to July 31st, 2019.

In [5]:
bbox = (7.364515, 46.868545, 7.535312, 46.999127)

In [10]:
item_search = service.search(collections=['ch.swisstopo.swisseo_s2-sr_v100'],
                             bbox=bbox,
                             datetime='2018-08-01/2019-07-31')

In [11]:
items = list(item_search.items())
print(f"Found {len(items)} items")

Found 124 items


Now we want to filter for items, which are at least 20% cloudfree. Since <em>SwissEO</em> is in `EPSG 2056`, we need first to transform th `bbox` accordingly

In [16]:
from pyproj import Transformer

# Create a transformer object
transformer = Transformer.from_crs("EPSG:4326", "EPSG:2056", always_xy=True)

# Transform the coordinates
x_min, y_min = transformer.transform(bbox[0], bbox[1])
x_max, y_max = transformer.transform(bbox[2], bbox[3])

# Create the new bounding box in EPSG:2056
bbox_2056 = (x_min, y_min, x_max, y_max)

No we create a function to test if an item is cloudfree enough. The function with `rasterio` will stream each `mask_10m.tif` file and check the second band if the values in the `bbox_2056` do at least have at least 20% 0 values - cloudfree pixels

In [17]:
import rasterio
import numpy 
def is_cloud_free_enough(item, bbox_2056, threshold=0.2):
    assets = item.assets
    asset_key_mask = next((key for key in assets.keys() if key.endswith('masks-10m.tif')), None)
    
    if asset_key_mask is None:
        return False
    
    with rasterio.open(assets[asset_key_mask].href) as src:
        window = src.window(*bbox_2056)
        cloud = src.read(2, window=window)
        
        # Calculate the percentage of cloud-free pixels (0 values)
        cloud_free_percentage = numpy.sum(cloud == 0) / cloud.size
        
        return cloud_free_percentage >= threshold

Now let's filter  our items with this function, since it will take a while to scan through the plethora of data we will use a prgress bar

In [ ]:
from tqdm import tqdm
filtered_items = []
for item in tqdm(items, desc="Checking items", unit="item"):
    if is_cloud_free_enough(item, bbox_2056):
        filtered_items.append(item)

print(f"Original number of items: {len(items)}")
print(f"Number of items with at least 20% cloud-free area: {len(filtered_items)}")

We have now our mostly cloudfree items in `filtered_items`

# Spectral Indices
<hr style="border:1px solid #0077b9;">

Spectral indices are computed using sensor bands to highlight a certain feature of a target or reduce certain effects. In this context, vegetation indices are spectral indices that enhance characteristics of vegetation, using bands such as Near Infra-red (`NIR`), a region where vegetation shows the most intense reflectance and bands located in red, where the vegetation has the highest absorption of visible sunlight due to the presence in its constitution of the green pigment chlorophyll (Meneses, 2012). The behavior of these spectral bands in some types of targets can be observed below:

<center>
<img src="https://brazil-data-cube.github.io/_images/spectral-profile.png" width="480" />
<br/>
Spectral profile of several targets. Source: modified from Pan et. al (2015).
</center>

# Calculating the Normalized Difference Vegetation Index (NDVI)
<hr style="border:1px solid #0077b9;">

The normalized difference vegetation index (NDVI) is calculated using the **Red** and **Near Infrared** (NIR) spectral bands. It assesses whether or not the target being observed contains live green vegetation. It can be calculated through the following equation:

$$
NDVI = \frac{(NIR - RED)}{(NIR + RED)}
$$

<center><b>Equation 1</b> - NDVI.</center>

We are going to compute the NDVI just with images from the first item:

In [19]:
#items = list(item_search.items())

In [ ]:
item = filtered_items[0]
item

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>Note:</b> <em>SwissEO</em> already provides the <em>VHI</em> alongside with the spectral bands for the item, besides the quality indicators (<em>MASKS</em>, <em>CLOUD</em>, <em>CLOUDPROBABILITY</em>, <em>CLOUD&TERRAINSHADOW</em>).
</div>

The assets with the links to the images, thumbnails or specific metadata files, can be accessed through the property `assets` (from a given item):

In [22]:
assets = item.assets

The data of the Sentinel-2 10 bands is available under the dictionary key which do end with`bands-10m.tif`:

In [23]:
asset_key_10m = next((key for key in assets.keys() if key.endswith('bands-10m.tif')), None)
band10m_asset = assets[asset_key_10m]


The metadata of the Sentinel-2 10 bands is available under the dictionary key which do end with`metadata.json`:

In [ ]:
asset_key_metadata = next((key for key in assets.keys() if key.endswith('metadata.json')), None)
metadata_asset = assets[asset_key_metadata ]
print(metadata_asset.href)

As one can see in the above linked json, the `B04`red wavelength  correspond to the the first band 1 in  'bands-10m.tif' and `B08` near-infrared to the band 4.

Now we retrieve with `rasterio` method the part of an image according to a rectangle specified in `bbox_2056`. We get then `red` and `nir` bands, which we scale back to radiance by factor 10'000:

In [38]:
with rasterio.open(assets[asset_key_10m].href) as src:
    window = src.window(*bbox_2056)   
    red = src.read(1, window=window)/10000  
    nir = src.read(4, window=window)/10000  

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>Note:</b> If there are errors because of your pyproj version, you can run the code below as specified in <a  href="https://rasterio.readthedocs.io/en/latest/faq.html#why-can-t-rasterio-find-proj-db-rasterio-from-pypi-versions-1-2-0" target="_blank">rasterio documentation</a> and try again:

       import os
       del os.environ['PROJ_LIB']
</div>

In [ ]:
red

In [ ]:
nir

Let's take a look at the retrieved data:

In [27]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(red, cmap='gray');

In [ ]:
plt.imshow(nir, cmap='gray');

Finally, let's compute the NDVI:

In [ ]:
ndvi = (nir - red)/(nir + red)
ndvi

Use the Matplotlib to visualize the result array, we subpress values lowert than 0, since they represent water:

In [ ]:
plt.imshow(ndvi, cmap='gray',vmin=0, vmax=1);

We can also plot using other colormaps:

In [ ]:
plt.imshow(ndvi, cmap='jet',vmin=0, vmax=1);

For more colormaps check https://matplotlib.org/stable/tutorials/colors/colormaps.html

# Histogram
<hr style="border:1px solid #0077b9;">

The histogram of a digital image, also known as frequency distribution, is the graphic representation in columns showing the intensity of values and the number of pixels with such intensity and is the basis for several types of digital image processing (Gonzalez & Woods, 2007). Some types of histogram can be observed below:

<center>
<img src="https://brazil-data-cube.github.io/_images/histogram.png" width="480" />
<br/>
Four types of images: dark, bright, low contrast and high contrast, and their respective histograms. Source: (Gonzalez & Woods, 2007).
</center>

# Thresholding images
<hr style="border:1px solid #0077b9;">

Digital image Classification is a broad topic and basically consists of assigning labels to targets in a dataset, also called classes. Although many of the classification approaches are complex, there are some simple approaches to solving certain problems, such as thresholding.

The idea of assigning a class following a threshold assumes that the data can be separated by a simple "line", this can be seen in the next figures in a histogram:


<center>
<img src="https://scikit-image.org/docs/dev/_images/sphx_glr_plot_multiotsu_001.png" width="960" />
<br/>
Histogram limiarization example. Source: <a href="https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_multiotsu.html">scikit-image doc</a>
</center>

Let's try to separate our data into groups according to their NDVI values.
But first let's see how the image histogram behaves:

In [ ]:
plt.title("NDVI Histogram")
plt.hist(ndvi)
plt.show()

Supposing we can separate the `ndvi` image with threshold, we would assume for this specific case that:
* all pixels with values below 0.2 are dark pixels;
* all pixels above 0.45 are areas containing a good portion of vegetation.
* all pixels with values from 0.2 to 0.45 are areas with few vegetation;

We can perform this thresholding by selecting in the ndvi matrix all values belonging to a given range and assigning a common integer value. We assume the following integer values:
* `1`: dark pixels;
* `2`: vigorous vegetation;
* `3`: weak vegetation.

Let's first create a copy of the original ndvi matrix:

In [46]:
labeled_img = ndvi.copy()

Now, we can use the new copy of the ndvi array and assign the values according to each range of values:

In [ ]:
labeled_img[ndvi < 0.2] = 1 # < 0.2
labeled_img[ndvi >= 0.2] = 3 # 0.2 - 0.45
labeled_img[ndvi >= 0.45] = 2 # >= 0.45
labeled_img

Finally, let's see the `ndvi` image separated into those labels:

In [ ]:
plt.rcParams['figure.figsize'] = [30, 20] #Change plot size
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(ndvi, cmap='gray')
ax2.imshow(labeled_img, cmap='brg');

# Calculating Image Difference
<hr style="border:1px solid #0077b9;">

Now let's suppose we want to compare the VHI for images from two different dates and same location, for instance to verify the areas where crops have grown and areas that loss vegetation.

For this computation we are going to use the VHI calculated from <em>SwissEO VHI</em>, and we will select two items (with the same location but with different dates) using STAC items.

The first image comprises pixels from July 28th, 2019 to August 12th, 2019 (`2019-07-28_2019-08-12`):

In [6]:
item_search = service.search(collections=['ch.swisstopo.swisseo_s2-sr_v100'],
                             bbox=bbox,
                             datetime='2024-08-01/2023-07-31')

In [7]:
items = list(item_search.items())
print(f"Found {len(items)} items")

Found 0 items


In [ ]:
filtered_items


In [ ]:
first_item = item

In [ ]:
first_item

In [ ]:
ndvi_first_image = read(first_item.assets['NDVI'].href, bbox=bbox)

The other selected image comprises pixels from `2018-07-28_2018-08-12` - i.e., one year before the first one selected:

In [ ]:
second_item = items[23]

In [ ]:
second_item

In [ ]:
ndvi_second_image = read(second_item.assets['NDVI'].href, bbox=bbox)

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>Note:</b> NDVI bands precomputed by BDC ranges from <em>-10000</em> to <em>10000</em>, instead of <em>-1</em> to <em>1</em>, as can be seen in the item metadata. This is due to the lower volume required to store files that use 16-bit integer values rather than 32-bit float.
</div>

Considering that these images are from an agricultural area and that crops are normally planted near August (first observation), in a six months, previous or after the first observation, it is expected to find crops, which will imply in greater NDVI values (more vigorous vegetation). This will cause NDVI band to present brighter values on these areas. Using the `gray` colormap, high value NDVI pixels will be more similar to white, while low value NDVI pixels will be closer to the black color.
Based on that, let's visually compare both NDVI images:

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(ndvi_first_image, cmap='gray')
ax2.imshow(ndvi_second_image, cmap='gray')

Assuming we want to see what have grown and what was loss, let's subtract the most recent image from the oldest one and plot it:

In [ ]:
ndvi_diff = ndvi_first_image - ndvi_second_image

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]
plt.imshow(ndvi_diff, cmap='jet');

As can be seen in the the NDVI difference plot, the main changes on pixel values were found in agriculture areas, which was expected due to changes in crops.
The blue values indicate negative values, while red values are positive. This means that for the blue areas there was a loss of vegetation, as a decreasing result on the NDVI value, meaning that crops were harvest. Meanwhile, on the red areas, the NDVI value increased as a result of the more vigorous vegetation on the recent date.

# References
<hr style="border:1px solid #0077b9;">

- [Python Client Library for STAC Service](https://pystac-client.readthedocs.io/en/latest/)

- [Spatio Temporal Asset Catalog Specification](https://stacspec.org/)

# See also the following Jupyter Notebooks
<hr style="border:1px solid #0077b9;">

* [Introduction to the SpatioTemporal Asset Catalog (STAC)](./stac-introduction.ipynb)